In [71]:
#!value --name example01
467..114..
...*......
..35...633
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664..598.

In [87]:
#!share --from value example01

type Part = {
    row:int
    head:int
    tail:int
    value:string
}
module Part =
    let create row head value =
        { row = row; head = head; tail = head; value = string value }
    let add tail value part =
        { part with tail = tail; value = part.value + string value }

type Symbol = {
    value: char
    row:int
    col:int
}

type Row = {
    col: int
    last: Part option
    parts: Part seq
    symbols: Symbol seq
}
module Row =
    let parse num (row:string) =
        let result =
            row.Trim().ToCharArray()
            |> Seq.fold (fun (state:Row) it ->
                match it with
                | digit when Char.IsDigit(digit) ->
                    { state with 
                        col = state.col + 1
                        last = state.last |> Option.map( Part.add state.col digit ) |> Option.orElse( (Part.create num state.col digit |> Some) )
                    }
                | '.' -> 
                    { state with 
                        col = state.col + 1
                        last = None
                        parts = match state.last with Some part -> state.parts |> Seq.append [ part ] | None -> state.parts 
                    }
                | symbol -> 
                    { state  with 
                        col = state.col + 1
                        last = None
                        parts = match state.last with Some part -> state.parts |> Seq.append [ part ] | None -> state.parts
                        symbols = state.symbols |> Seq.append [ { col = state.col; row = num; value = symbol } ]
                    }
            ) { col = 0; last = None; parts = Seq.empty; symbols = Seq.empty }
        { result with 
            parts = match result.last with Some part -> result.parts |> Seq.append [ part ] | None -> result.parts } 


let parse lines =
    lines
    |> Seq.mapi Row.parse
    |> Seq.fold (fun (parts, symbols) row -> 
        ( parts |> Seq.append row.parts) , ( symbols |> Seq.append row.symbols )
    ) (Seq.empty, Seq.empty)

let filterValid (parts, symbols) =
    symbols
    |> Seq.map(fun symbol ->
        parts
        |> Seq.filter( fun (part:Part) ->
            match part with
            | { head = h; tail = t; row = r; value = _ } when r = symbol.row ->
                h = symbol.col + 1 || t = symbol.col - 1
            | { head = h; tail = t; row = r; value = _ } when r = symbol.row + 1 || r = symbol.row - 1 ->
                h <= symbol.col + 1 && t >= symbol.col - 1
            | _ -> false
        )
    )
    |> Seq.concat

example01.Split("\n") 
|> parse
|> filterValid
|> Seq.sumBy (fun part -> int part.value)

4361

In [88]:
let ResolutionFolder = __SOURCE_DIRECTORY__
File.ReadAllText(ResolutionFolder + "/input01.txt").Split("\n")
|> parse
|> filterValid
|> Seq.sumBy (fun part -> int part.value)

525181

In [94]:
#!share --from value example01

let filterRatios (parts,symbols) =
    symbols
    |> Seq.filter (fun symbol -> symbol.value = '*')
    |> Seq.map(fun symbol ->
        parts
        |> Seq.filter( fun (part:Part) ->
            match part with
            | { head = h; tail = t; row = r; value = _ } when r = symbol.row ->
                h = symbol.col + 1 || t = symbol.col - 1
            | { head = h; tail = t; row = r; value = _ } when r = symbol.row + 1 || r = symbol.row - 1 ->
                h <= symbol.col + 1 && t >= symbol.col - 1
            | _ -> false
        )
        |> Seq.map(fun part -> int part.value)
    )
    |> Seq.filter(fun parts -> (parts |> Seq.length) = 2 )

example01.Split("\n")
|> parse
|> filterRatios
|> Seq.map ( Seq.reduce (*))
|> Seq.sum

467835

In [95]:
let ResolutionFolder = __SOURCE_DIRECTORY__
File.ReadAllText(ResolutionFolder + "/input01.txt").Split("\n")
|> parse
|> filterRatios
|> Seq.map ( Seq.reduce (*))
|> Seq.sum

84289137